In [3]:
import torch
import numpy as np
import nir
import json

In [8]:
graph = nir.read("braille_subtract.nir")

In [10]:
graph.nodes.keys()

dict_keys(['fc1', 'fc2', 'input', 'lif1.lif', 'lif1.w_rec', 'lif2', 'output'])

In [11]:
graph.nodes["lif1.w_rec"].weight.shape

(55, 55)

In [22]:
def rework_braille_graph(graph):
    graph = nir.read(graph)
    hidden = np.array([graph.nodes["lif1.w_rec"].weight.shape[0]])
    subgraph = nir.NIRGraph(
        nodes = {"input": nir.Input(hidden), "lif1": graph.nodes["lif1.lif"], "lin": graph.nodes["lif1.w_rec"], "output": nir.Output(hidden)},
        edges = [("input", "lif"), ("lif", "lin"), ("lin", "lif"), ("lif", "output")]
    )
    new_graph = nir.NIRGraph(
        nodes = {"input": graph.nodes["input"], "fc1": graph.nodes["fc1"], "fc2": graph.nodes["fc2"], "lif1": subgraph, "lif2": graph.nodes["lif2"], "output": graph.nodes["output"]},
        edges = [("input", "fc1"), ("fc1", "lif1"), ("lif1", "fc2"), ("fc2", "lif2"), ("lif2", "output")]
    )
    return new_graph
new_subtract = rework_braille_graph("braille_subtract.nir")
nir.write("braille_subtract_subgraph.nir", new_subtract)
new_zero = rework_braille_graph("braille_zero.nir")
nir.write("braille_zero_subgraph.nir", new_zero)

In [19]:
import norse.torch.utils as nu
nu.from_nir(new_graph)(torch.empty(12))

Please read the Norse documentation for more information.
This warning can be turned off by setting from_nir(..., ignore_warnings=True)

Please read the Norse documentation for more information.
This warning can be turned off by setting from_nir(..., ignore_warnings=True)



(tensor([0., 0., 0., 0., 0., 0., 0.], grad_fn=<SuperSpikeBackward>),
 GraphExecutorState(state={'lif1': (RecurrentSequentialState(cache=tensor([-0.5638, -0.5395, -0.5250, -0.4674, -0.0355, -0.5286, -0.3670, -0.0750,
          0.0210, -0.0689, -0.0777, -0.2457, -0.1274,  0.0114, -1.0532, -0.3147,
         -0.4618, -0.9926, -0.6294,  0.0518,  0.0164, -0.8293,  0.0284,  0.0717,
         -0.5576, -0.1293, -0.0519, -0.7355, -0.3816, -0.0711, -0.0503, -0.0018,
         -0.4985, -0.0235, -0.8448,  0.0581, -0.0813, -0.1282, -0.4182, -0.2181,
         -0.1173, -0.3772, -0.7924, -0.8783, -0.0159, -0.4439, -0.0940, -0.5002,
         -0.6602, -0.9305, -0.6093, -0.0884, -0.0210, -0.0041, -0.1188],
        grad_fn=<ViewBackward0>), state=[LIFFeedForwardState(v=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0

In [ ]:
nir.write("braille")

In [2]:
 m = torch.load("graph_Braille_snntorch_20231005_140408.pt", map_location="cpu")
m = torch.load("data/model_ref_20231011_100043.pt", map_location="cpu")
m

OrderedDict([('fc1.weight',
              tensor([[-2.4941e-01, -2.7045e-01, -5.6922e-01, -4.4093e-01, -6.2774e-01,
                       -4.1964e-01, -5.8216e-01, -3.3659e-01, -2.2998e-01, -6.1245e-01,
                       -2.5483e-01, -3.6886e-01],
                      [ 5.0665e-01, -2.4508e-01, -2.6034e-01,  1.5704e-01,  4.0923e-01,
                        6.6307e-02, -5.1929e-01, -1.8984e-01,  4.5717e-01, -1.7989e-03,
                       -3.1230e-01, -5.1122e-01],
                      [-2.5532e+00, -2.1951e-01,  2.4656e-04, -1.1556e-01,  1.1486e-01,
                        1.5903e-01, -4.9985e-02, -3.1966e+00, -9.3085e-01, -3.7166e-01,
                        1.7233e-01, -2.8365e-02],
                      [-4.5669e-01, -9.2637e-02, -2.6358e-01, -4.9646e-01, -3.8727e-01,
                       -4.8542e-01, -1.8541e-01, -6.9954e-02, -3.3106e-01, -5.4669e-01,
                       -1.8387e-01, -3.8676e-01],
                      [ 5.3529e-01, -1.4933e-01, -7.3964e-01, -7.126

In [3]:
[f"{x:25s}: {m[x].shape}" for x in m.keys()]

['fc1.weight               : torch.Size([38, 12])',
 'fc1.bias                 : torch.Size([38])',
 'lif1.threshold           : torch.Size([])',
 'lif1.graded_spikes_factor: torch.Size([])',
 'lif1.reset_mechanism_val : torch.Size([])',
 'lif1.beta                : torch.Size([])',
 'lif1.alpha               : torch.Size([])',
 'lif1.recurrent.weight    : torch.Size([38, 38])',
 'lif1.recurrent.bias      : torch.Size([38])',
 'fc2.weight               : torch.Size([7, 38])',
 'fc2.bias                 : torch.Size([7])',
 'lif2.threshold           : torch.Size([])',
 'lif2.graded_spikes_factor: torch.Size([])',
 'lif2.reset_mechanism_val : torch.Size([])',
 'lif2.beta                : torch.Size([])',
 'lif2.alpha               : torch.Size([])']

In [4]:
m["lif1.recurrent.bias"]

tensor([-0.4165,  0.0111,  0.1997, -0.5325,  0.1633, -0.5052,  0.0697, -1.4138,
        -1.9939, -2.0510,  0.1497, -0.9479, -0.1543, -0.3534,  0.0114, -0.4838,
        -0.5493, -0.4289, -2.6824, -0.0507, -1.4944, -2.2189, -0.0953,  0.0425,
         0.0493, -1.5376, -0.5980, -0.0681, -1.3593, -0.2271, -1.3662, -0.3498,
         0.0173, -0.1951, -0.2375, -0.8413, -0.1211, -0.7390])

In [5]:
m["lif1.graded_spikes_factor"]

tensor(1.)

In [6]:
### OPTIMAL HYPERPARAMETERS
parameters_path = "data/parameters_ref_20231011.json"

with open(parameters_path) as f:
   parameters = json.load(f)

parameters

{'nb_hidden': 38,
 'alpha_r': 0.55,
 'alpha_out': 0.5,
 'beta_r': 0.9,
 'beta_out': 0.55,
 'lr': 0.005,
 'reg_l1': 0.0006,
 'reg_l2': 4e-06,
 'slope': 5}

In [11]:
to_array = lambda v, s: np.ones(s) * np.array(v)
n_inputs = 12
n_hidden = parameters["nb_hidden"]
n_out = 7
sub = nir.NIRGraph(
    nodes = {"input": nir.Input(np.array([n_inputs])),
             "lif": nir.CubaLIF(tau_syn=to_array(1 / (1 - m["lif1.beta"]), n_hidden), tau_mem=to_array(1 / (1 - m["lif1.alpha"]), n_hidden),
                                r=to_array(1, n_hidden), v_leak=to_array(0, n_hidden), v_threshold=to_array(m["lif1.threshold"], n_hidden),
                                w_in=to_array(1, n_hidden)),
             "lin": nir.Affine(weight=m["lif1.recurrent.weight"], bias=m["lif1.recurrent.bias"]),
             "output": nir.Output(np.array([n_hidden]))},
    edges = [("input", "lif"), ("lif", "lin"), ("lin", "lif"), ("lif", "output")]
)

In [12]:
g = nir.NIRGraph(
    nodes = {
        "input": nir.Input(np.array([n_inputs])),
        "fc1": nir.Affine(weight=m["fc1.weight"], bias=m["fc1.bias"]),
        "fc2": nir.Affine(weight=m["fc2.weight"], bias=m["fc2.bias"]),
        "lif2": nir.CubaLIF(tau_syn=to_array(1 / (1 - m["lif2.beta"]), n_out), tau_mem=to_array(1 / (1 - m["lif2.alpha"]), n_out),
                            r=to_array(1, n_out), v_leak=to_array(0, n_out), v_threshold=to_array(m["lif2.threshold"], n_out),
                            w_in=to_array(1, n_out)),
        "hidden": sub,
        "output": nir.Output(np.array([n_out]))
    },
    edges = [("input", "fc1"), ("fc1", "hidden"), ("hidden", "fc2"), ("fc2", "lif2"), ("lif2", "output")]
)

nir.write("braille_subgraph.nir", g)

In [13]:
g = nir.read("braille_subgraph.nir")
g

NIRGraph(nodes={'fc1': Affine(weight=array([[-2.49409378e-01, -2.70445287e-01, -5.69223404e-01,
        -4.40930456e-01, -6.27739668e-01, -4.19635981e-01,
        -5.82160652e-01, -3.36590886e-01, -2.29984209e-01,
        -6.12446606e-01, -2.54827499e-01, -3.68863791e-01],
       [ 5.06653666e-01, -2.45075211e-01, -2.60343432e-01,
         1.57037631e-01,  4.09227639e-01,  6.63074553e-02,
        -5.19294858e-01, -1.89839453e-01,  4.57174689e-01,
        -1.79891742e-03, -3.12303036e-01, -5.11221111e-01],
       [-2.55315304e+00, -2.19514430e-01,  2.46555923e-04,
        -1.15561672e-01,  1.14861339e-01,  1.59026250e-01,
        -4.99848351e-02, -3.19663167e+00, -9.30854976e-01,
        -3.71663630e-01,  1.72332600e-01, -2.83649899e-02],
       [-4.56685811e-01, -9.26365629e-02, -2.63577610e-01,
        -4.96459752e-01, -3.87274384e-01, -4.85415697e-01,
        -1.85405463e-01, -6.99538589e-02, -3.31062198e-01,
        -5.46691954e-01, -1.83872759e-01, -3.86764467e-01],
       [ 5.3528